## 3-Body simulation

In [1]:
import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

class Body:
    """Represents a body with methods to compute forces and displacements."""

    def __init__(self, x, y, radius, color, mass, vx=0, vy=0):
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.mass = mass
        self.vx = vx
        self.vy = vy
        self.orbit = [(x, y)]


In [2]:
def update_position(body, dt):
    body.x += body.vx * dt
    body.y += body.vy * dt

def update_velocity(body, force, dt):

    ax = force[0] / body.mass
    ay = force[1] / body.mass
    body.vx += ax * dt
    body.vy += ay * dt


In [4]:
def calculate_kinetic_energy(body):
    """Calculate kinetic energy of a body"""
    v_squared = body.vx**2 + body.vy**2
    return 0.5 * body.mass * v_squared

def calculate_potential_energy(body1, body2):
    """Calculate gravitational potential energy between two bodies"""
    G = 1.0  # matching your simulation
    dx = body2.x - body1.x
    dy = body2.y - body1.y
    distance = math.sqrt(dx**2 + dy**2)

    # Gravitational potential energy (negative)
    return -G * body1.mass * body2.mass / distance

def check_escape_energy(test_body, other_bodies, energy_offset=2.0):
    """
    Check if a body has enough energy to escape from the others.
    Returns True if the body is escaping, False otherwise.
    """
    # Calculate kinetic energy
    kinetic_energy = calculate_kinetic_energy(test_body)

    # Calculate total potential energy with all other bodies
    potential_energy = 0
    for other_body in other_bodies:
        if other_body != test_body:
            potential_energy += calculate_potential_energy(test_body, other_body)

    # Total mechanical energy
    total_energy = kinetic_energy + potential_energy
    #if total_energy > 0:
      #print(f"Total energy: {total_energy}")
    # If total energy is positive, the body can escape
    return total_energy > energy_offset

def check_all_bodies_escape(bodies):
    """
    Check escape status for all bodies in the system.
    Returns a dictionary with escape status for each body.
    """
    escape_status = {}

    for i, body in enumerate(bodies):
        is_escaping = check_escape_energy(body, bodies)
        escape_status[f"body_{i}"] = {
            "body": body,
            "is_escaping": is_escaping,
            "color": body.color
        }

    return escape_status

def classify_system_state(bodies, dt, max_steps=1000, collision_threshold=0.1):
    """
    Classify the system as diverging, converging, or stable based on escape analysis
    and collision detection over a given time frame.

    Returns: "diverging", "converging", or "stable"
    """
    # Make a deep copy of bodies to avoid modifying the original
    import copy
    test_bodies = [copy.deepcopy(body) for body in bodies]

    # Track initial escape status
    initial_escape = check_all_bodies_escape(test_bodies)
    any_escaping = any(status["is_escaping"] for status in initial_escape.values())

    # If any body is already escaping at the start, classify as diverging
    if any_escaping:
        return "diverging"

    # Simulate forward in time to check for collisions
    for step in range(max_steps):
        # Check for collisions
        for i in range(len(test_bodies)):
            for j in range(i + 1, len(test_bodies)):
                dx = test_bodies[i].x - test_bodies[j].x
                dy = test_bodies[i].y - test_bodies[j].y
                distance = math.sqrt(dx**2 + dy**2)

                if distance < collision_threshold:
                    return "converging"

        # Simulate one step
        simulate(test_bodies, dt)

    # No collision or escape detected within time frame
    return "stable"

# Example usage in your simulation:
def analyze_current_state(bodies):
    """Analyze the current state of the system"""
    escape_status = check_all_bodies_escape(bodies)

    for body_id, status in escape_status.items():
        if status["is_escaping"]:
            print(f"{status['color']} body is escaping!")
        else:
            print(f"{status['color']} body is bound")

    # Classify the overall system
    system_state = classify_system_state(bodies, dt=0.01, max_steps=500)
    print(f"System classification: {system_state}")

    return system_state

In [5]:

def detect_collisions(bodies, prev_positions):
    """Detects collisions between bodies considering their paths and intermediate points."""
    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            body1 = bodies[i]
            body2 = bodies[j]

            # Previous positions
            prev_x1, prev_y1 = prev_positions[body1]
            prev_x2, prev_y2 = prev_positions[body2]
            distance = math.sqrt((body1.x - body2.x)**2 + (body1.y - body2.y)**2)

            # Check for collision using line segment intersection
            if line_segments_intersect(prev_x1, prev_y1, body1.x, body1.y,
                                      prev_x2, prev_y2, body2.x, body2.y):
                return True, distance, body1, body2  # Collision detected

            # Check for collision at intermediate points
            num_intermediate_points = 3
            for k in range(1, num_intermediate_points + 1):
                intermediate_x1 = prev_x1 + (body1.x - prev_x1) * (k / (num_intermediate_points + 1))
                intermediate_y1 = prev_y1 + (body1.y - prev_y1) * (k / (num_intermediate_points + 1))
                intermediate_x2 = prev_x2 + (body2.x - prev_x2) * (k / (num_intermediate_points + 1))
                intermediate_y2 = prev_y2 + (body2.y - prev_y2) * (k / (num_intermediate_points + 1))

                distance = math.sqrt((intermediate_x1 - intermediate_x2)**2 + (intermediate_y1 - intermediate_y2)**2)
                if distance <= body1.radius + body2.radius:
                    return True, distance, body1, body2  # Collision detected at intermediate point

    return False, 100, None, None  # No collision detected

def line_segments_intersect(x1, y1, x2, y2, x3, y3, x4, y4):
    """Checks if two line segments intersect."""
    # Calculate orientations
    d1 = (x2 - x1) * (y3 - y4) - (y2 - y1) * (x3 - x4)
    if d1 == 0:  # Lines are parallel or collinear
        return False  # Treat as no intersection for simplicity

    t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / d1
    u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / d1

    return 0 <= t <= 1 and 0 <= u <= 1  # Intersect within segment bounds

In [7]:
def gravitational_force(body1, body2, G):

    # Displacements.
    # G = 1.34
    dx = body2.x - body1.x
    dy = body2.y - body1.y

    # Distances.
    distance_squared = dx**2 + dy**2
    distance = math.sqrt(distance_squared)

    # Forces.
    force_magnitude = G * body1.mass * body2.mass / distance_squared
    force_x = force_magnitude * dx / distance
    force_y = force_magnitude * dy / distance

    return (force_x, force_y)

def simulate(bodies, dt, G):

    # Store previous positions
    prev_positions = {body: (body.x, body.y) for body in bodies}
    global time
    global time_step
    time += dt
    time_step += 1

    # Loop through each body.
    for body in bodies:

        # Sum forces.
        force_sum = [0, 0]
        for other_body in bodies:
            if body != other_body:
                force = gravitational_force(body, other_body, G)
                force_sum[0] += force[0]
                force_sum[1] += force[1]

        update_velocity(body, force_sum, dt)
    for body in bodies:
        update_position(body, dt)
        body.orbit.append((body.x, body.y))
    escape = False
    escape_detected = check_all_bodies_escape(bodies)
    for status in escape_detected.values():
        if status["is_escaping"]:
            #print(f"{status['color']} body is escaping at Time step: {time_step}, Time: {time:.2f}!")
            escape = True

    # Collision detection
    collision_detected, distance, body1, body2 = detect_collisions(bodies, prev_positions)
    #if collision_detected:
        #print(f"Collision detected between {body1.color} and {body2.color} body for distance: {distance} at Time step: {time_step}, Time: {time:.2f}!")
        # Handle collision (e.g., merge bodies, stop simulation)


    return collision_detected, distance, escape



In [8]:
from re import escape
# Animation code.
def animate(frame):
    global bodies
    dt = 0.01
    G = 1.0 # default 1.0
    collide, distance, escape = simulate(bodies, dt, G)
    ax.clear()

    # Agregar líneas de ejes cartesianos
    ax.axhline(0, color='black',linewidth=0.5)
    ax.axvline(0, color='black',linewidth=0.5)
    # Add text for time and time step
    time_text = ax.text(0.05, 0.95, f'Time: {time:.2f}', transform=ax.transAxes)
    timestep_text = ax.text(0.05, 0.90, f'Time step: {time_step}', transform=ax.transAxes)
    if collide:
        collide_text = ax.text(0.05, 0.85, f'Collision occurred: {collide} at distance: {distance}', transform=ax.transAxes)
    if escape:
        escape_text = ax.text(0.05, 0.80, f'Escape occurred: {escape}', transform=ax.transAxes)

    # Plot bodies
    for body in bodies:
        ax.scatter(
            body.x,
            body.y,
            color=body.color,
            s=100,
            label=f'{body.color} body'
        )
        updated_points = list(zip(*body.orbit))
        ax.plot(updated_points[0], updated_points[1], color=body.color, linewidth=2)



In [ ]:
import matplotlib

# Initial values for simulation:
v = 1.0
L = 1.0
time_step = 0
time = 0

v1x = v
v1y = 0
v2x = -v / 2
v2y = v * math.sqrt(3) / 2
v3x = -v / 2
v3y = -v * math.sqrt(3) / 2

# x, y, radius, color, mass, vx, vy
adjust_mass = math.sqrt(1.4799)
body_A = Body(-L/2, 0, 0.02, "red", 2 * adjust_mass, v1x, v1y)
body_B = Body(L/2, 0, 0.02, "green", 2 * adjust_mass, v2x, v2y)
body_C = Body(0, L * math.sqrt(3) / 2, 0.02, 'blue', 1 * adjust_mass, v3x, v3y)

bodies = [body_A, body_B, body_C]

# Data structures to store information
positions = {body: [] for body in bodies}
velocities = {body: [] for body in bodies}
escape_statuses = []
collision_occurred = False  # Flag to track if a collision happened
# Store data
for body in bodies:
    positions[body].append((body.x, body.y))
    velocities[body].append((body.vx, body.vy))

# Simulation loop
dt = 0.01
G = 1.0
num_steps = 900  # Adjust as needed
for step in range(num_steps):
    collision_occurred, _, escape = simulate(bodies, dt, G)
    # Store data
    for body in bodies:
        positions[body].append((body.x, body.y))
        velocities[body].append((body.vx, body.vy))
    if collision_occurred:
        print("Collision detected! Stopping simulation.")
        break  # Stop if collision occurs
    if escape:
        print("Escape detected! Stopping simulation.")
        break  # Stop if escape occurs


Total energy: 7.949157757198314
blue body is escaping at Time step: 807, Time: 8.07!
Escape detected! Stopping simulation.


In [9]:
import numpy as np
import itertools
import copy
import pickle
from datetime import datetime

def create_test_data(base_velocities, velocity_range, num_increments,
                    simulation_params, output_file='3body_test_data.pkl'):
    """
    Generate test data by varying velocities of three bodies and running simulations.

    Parameters:
    - base_velocities: dict with keys 'v1x', 'v1y', 'v2x', 'v2y', 'v3x', 'v3y'
    - velocity_range: float, range of velocity variation (e.g., 0.2 for ±20%)
    - num_increments: int, number of increments in each direction
    - simulation_params: dict with 'L', 'dt', 'num_steps', 'collision_threshold'
    - output_file: string, filename to save the data

    Returns:
    - test_data: list of dictionaries containing simulation results
    """

    # Set time to zero
    time_step = 0
    time = 0

    # Extract parameters
    G = simulation_params['G']
    L = simulation_params['L']
    dt = simulation_params['dt']
    num_steps = simulation_params['num_steps']
    collision_threshold = simulation_params['collision_threshold']

    # Create velocity increments
    velocity_variations = np.linspace(-velocity_range, velocity_range, num_increments)

    # Store all test data
    test_data = []
    simulation_count = 0

    # Generate all combinations of velocity variations
    velocity_keys = ['v1x', 'v1y', 'v2x', 'v2y', 'v3x', 'v3y']

    # For full factorial design (this will be a lot of simulations!)
    # For 6 velocities with n increments each, this is n^6 simulations
    for velocity_deltas in itertools.product(velocity_variations, repeat=6):
        simulation_count += 1

        # Create varied velocities
        varied_velocities = {}
        for i, key in enumerate(velocity_keys):
            varied_velocities[key] = base_velocities[key] * (1 + velocity_deltas[i])
            # print(f"Velocity for {key} is {varied_velocities[key]}")

        # Create bodies with varied velocities
        adjust_mass = math.sqrt(1.4799)
        body_A = Body(-L/2, 0, 0.02, "red", 2 * adjust_mass,
                     varied_velocities['v1x'], varied_velocities['v1y'])
        body_B = Body(L/2, 0, 0.02, "green", 2 * adjust_mass,
                     varied_velocities['v2x'], varied_velocities['v2y'])
        body_C = Body(0, L * math.sqrt(3) / 2, 0.02, 'blue', 1 * adjust_mass,
                     varied_velocities['v3x'], varied_velocities['v3y'])

        bodies = [body_A, body_B, body_C]

        # Data structures to store information
        positions = {body: [] for body in bodies}
        velocities = {body: [] for body in bodies}
        # Store data
        for body in bodies:
            positions[body].append((body.x, body.y))
            velocities[body].append((body.vx, body.vy))

        # Run simulation
        collision_step = None
        escape_step = None
        system_state = "stable"  # default

        for step in range(num_steps):
          collision_occurred, _, escape = simulate(bodies, dt, G)
          # Store data
          for body in bodies:
              positions[body].append((body.x, body.y))
              velocities[body].append((body.vx, body.vy))
          if collision_occurred:
              print("Collision detected! Stopping simulation.")
              collision_step = step
              system_state = "converging"
              break  # Stop if collision occurs
          if escape:
              print("Escape detected! Stopping simulation.")
              escape_step = step
              system_state = "diverging"
              break  # Stop if escape occurs


        # Store simulation results
        result = {
            'simulation_id': simulation_count,
            'initial_velocities': varied_velocities,
            'velocity_deltas': velocity_deltas,
            'positions': positions,
            'velocities': velocities,
            'system_state': system_state,
            'collision_step': collision_step,
            'escape_step': escape_step,
            'num_steps_simulated': max(len(positions[body]) for body in bodies)
        }

        test_data.append(result)

        # Progress report
        if simulation_count % 100 == 0:
            print(f"Completed {simulation_count} simulations...")

    # Save data
    with open(output_file, 'wb') as f:
        pickle.dump(test_data, f)

    print(f"Completed {simulation_count} simulations. Data saved to {output_file}")

    # Summary statistics
    states = {'converging': 0, 'diverging': 0, 'stable': 0}
    for result in test_data:
        states[result['system_state']] += 1

    print("\nSummary:")
    for state, count in states.items():
        print(f"{state}: {count} ({100*count/simulation_count:.1f}%)")

    return test_data

# Example usage:
def run_parameter_sweep(num_steps=900, num_increments=3, velocity_range=0.2):
    """Run a parameter sweep with reasonable defaults"""

    # Base velocities from your code
    v = 1.0
    base_velocities = {
        'v1x': v,
        'v1y': 0,
        'v2x': -v / 2,
        'v2y': v * math.sqrt(3) / 2,
        'v3x': -v / 2,
        'v3y': -v * math.sqrt(3) / 2
    }

    # Simulation parameters
    simulation_params = {
        'G': 1.0,
        'L': 1.0,
        'dt': 0.01,
        'num_steps': num_steps,
        'collision_threshold': 0.05
    }

    # Run with ±20% velocity variations, 5 increments per velocity
    # Note: 5^6 = 15,625 simulations!
    # For testing, you might want to start with fewer increments
    test_data = create_test_data(
        base_velocities=base_velocities,
        velocity_range=velocity_range,  # ±20%
        num_increments=num_increments,    # 3^6 = 729 simulations (more manageable)
        simulation_params=simulation_params,
        output_file='3body_test_data.pkl'
    )

    return test_data

# Function to load and analyze the data later
def load_test_data(filename='3body_test_data.pkl'):
    """Load test data from file"""
    with open(filename, 'rb') as f:
        test_data = pickle.load(f)
    return test_data

# Example of how to use the data for machine learning
def prepare_ml_dataset(test_data):
    """
    Convert test data into format suitable for machine learning.
    Returns X (features) and y (labels).
    """
    X = []  # Features: initial velocities
    y = []  # Labels: system state

    for result in test_data:
        # Extract initial velocities as features
        velocities = result['initial_velocities']
        features = [velocities[key] for key in ['v1x', 'v1y', 'v2x', 'v2y', 'v3x', 'v3y']]
        X.append(features)

        # Extract label
        label = result['system_state']
        y.append(label)

    return np.array(X), np.array(y)

In [10]:
!ls

sample_data


In [ ]:
# Run the parameter sweep
test_data = run_parameter_sweep(800, 6, 0.8)

In [ ]:
import matplotlib

# Initial values for simulation:
v = 1.0
L = 1.0
time_step = 0
time = 0

# x, y, radius, color, mass, vx, vy
adjust_mass = math.sqrt(1.4799)
body_A = Body(-L/2, 0, 0.02, "red", 2 * adjust_mass, v, 0)
body_B = Body(L/2, 0, 0.02, "green", 2 * adjust_mass, -v / 2, v * math.sqrt(3) / 2)
body_C = Body(0, L * math.sqrt(3) / 2, 0.02, 'blue', 1 * adjust_mass, -v / 2, -v * math.sqrt(3) / 2)

bodies = [body_A, body_B, body_C]

# Generate animation.
matplotlib.rcParams['animation.embed_limit'] = 100  # Set limit to 100 MB (or another desired value)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, animate, frames=range(900), interval=50)

player = HTML(ani.to_jshtml(fps=10)); plt.close()

# Call player to view the movie.
player


In [ ]:
import matplotlib
# Collides at step 647 for G = 1.34.
# Initial values for simulation:
v = 1.0
L = 1.0
time_step = 0
time = 0
body_A = Body(-L/2, 0, 0.02, "red", 2 * 1.16, v, 0)
body_B = Body(L/2, 0, 0.02, "green", 2 * 1.15, -v / 2, v * math.sqrt(3) / 2)
body_C = Body(0, L * math.sqrt(3) / 2 * 1.3511, 0.02, 'blue', 1, -v / 2, -v * math.sqrt(3) / 2)

bodies = [body_A, body_B, body_C]

# Generate animation.
matplotlib.rcParams['animation.embed_limit'] = 100  # Set limit to 100 MB (or another desired value)
fig, ax = plt.subplots()
ani = FuncAnimation(fig, animate, frames=range(700), interval=50)

player = HTML(ani.to_jshtml(fps=10)); plt.close()

# Call player to view the movie.
player
